In [1]:
import os
os.chdir('..')
%pwd

'c:\\Users\\larakim\\Documents\\Data_train\\Satellite_image_classif\\ML-pipeline-Satellite-image-classification'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class Model_trainer_config:
    root_dir: Path
    base_model_path: Path
    updated_model_path: Path
    data_path: Path
    epoch: int
    batch_size: int
    img_size: tuple
    lr:float

In [3]:
from img_classif.utils.common import read_yaml, create_directories
from img_classif.constants import *

class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH,param_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(param_path)

    def get_trainModel(self)->Model_trainer_config:
        updated_model_config = Model_trainer_config(
        root_dir = self.config.model_trainer.root_dir,
        base_model_path = self.config.base_model_prep.base_model_path,
        updated_model_path = self.config.model_trainer.updated_model_path,
        data_path = self.config.model_trainer.data_path,
        epoch = self.params.epochs,
        batch_size = self.params.batch_size,
        img_size=self.params.img_size,
        lr=self.params.lr
        )

        return updated_model_config

In [13]:
import tensorflow as tf
from img_classif import logger
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, average_precision_score, recall_score, precision_score, accuracy_score, classification_report
import keras

class Modeltrainer:
    def __init__(self, config: Model_trainer_config):
        self.config = config
        create_directories([self.config.root_dir])

    def train_model(self):
        """
        """
        # Loading model
        model = tf.keras.models.load_model(self.config.base_model_path)
        

      
        input_shape=eval(self.config.img_size)
        # Initialize the ImageDataGenerator (here, we're just rescaling the images)
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                fill_mode='nearest',
            validation_split=0.2) 

        ds_train = train_datagen.flow_from_directory(
            self.config.data_path,
            color_mode="rgb",
            batch_size=self.config.batch_size,
            shuffle=False,
            seed=10,
            subset='training',
            interpolation="nearest",
            target_size=input_shape[:2],
        )
        ds_val = train_datagen.flow_from_directory(
            self.config.data_path,
            color_mode="rgb",
            batch_size=self.config.batch_size,
            shuffle=False,
            seed=10,
            subset='validation',
            interpolation="nearest",
            target_size=input_shape[:2],
        )
        labels = dict((v,k) for k,v in (ds_val.class_indices).items())

        logger.info('In train_generator')
        for i in range(len (ds_train.class_indices)):
            print(labels[i],":\t",list(ds_train.classes).count(i))
   

        logger.info('In validation_generator')
        for i in range(len (ds_val.class_indices)):
            print(labels[i],":\t",list(ds_val.classes).count(i))

    
        # Define callbacks
        checkpoint = ModelCheckpoint(self.config.updated_model_path, verbose=1, monitor='val_loss', save_best_only=True, mode='auto')

        # EarlyStopping to stop training when the validation loss has not improved after 5 epochs
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

        # ReduceLROnPlateau to reduce the learning rate when the validation loss has stopped improving
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, mode='auto', min_lr=0.00001)

        # Start model training
        ds_train.reset()
        ds_val.reset()

        # Fit the model
        history = model.fit(
            ds_train,
            epochs=self.config.epoch,  # Adjust based on your needs
            validation_data=ds_val,
            callbacks=[checkpoint, early_stopping, reduce_lr]
        )
        logger.info('Saving trained model')
        model.save(self.config.updated_model_path)
        logger.info(f'Trained model saved at {self.config.updated_model_path}')


In [14]:
config = ConfigurationManager()
training_config = config.get_trainModel()
model_c = Modeltrainer(config=training_config)
model_c.train_model()

['2024-04-08 11:37:08,048':'INFO':'common':'config\config.yaml loaded successfully']
['2024-04-08 11:37:08,052':'INFO':'common':'params.yaml loaded successfully']
['2024-04-08 11:37:08,055':'INFO':'common':'created directory at: artifacts/updated_model']
Found 4505 images belonging to 4 classes.
Found 1126 images belonging to 4 classes.
['2024-04-08 11:37:10,591':'INFO':'3258207959':'In train_generator']
cloudy :	 1200
desert :	 905
green_area :	 1200
water :	 1200
['2024-04-08 11:37:10,594':'INFO':'3258207959':'In validation_generator']
cloudy :	 300
desert :	 226
green_area :	 300
water :	 300
 60/141 [===========>..................] - ETA: 1:08 - loss: 1.2153 - accuracy: 0.4914

KeyboardInterrupt: 

In [10]:
base_model.get_compile_config()

{'optimizer': {'module': 'keras.optimizers',
  'class_name': 'Adam',
  'config': {'name': 'Adam',
   'weight_decay': None,
   'clipnorm': None,
   'global_clipnorm': None,
   'clipvalue': None,
   'use_ema': False,
   'ema_momentum': 0.99,
   'ema_overwrite_frequency': None,
   'jit_compile': False,
   'is_legacy_optimizer': False,
   'learning_rate': 0.0010000000474974513,
   'beta_1': 0.9,
   'beta_2': 0.999,
   'epsilon': 1e-07,
   'amsgrad': False},
  'registered_name': None},
 'loss': 'categorical_crossentropy',
 'metrics': ['accuracy'],
 'loss_weights': None,
 'weighted_metrics': None,
 'run_eagerly': None,
 'steps_per_execution': None,
 'jit_compile': None}